In [418]:
import glob

genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = 26  # Which month and days result you want to visualize in table
month = 10

curSavePath = f"{genSavePath}{day}-{month}-5/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.6

In [419]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]

                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])

                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
table = pd.DataFrame(np.array(subjectResultArray), columns=headers)
display(table)


(1, 154)
(10, 154)


c:\Users\Luna\anaconda3\envs\mentalab3\lib\site-packages\ipykernel_launcher.py:170: RuntimeWarning: invalid value encountered in true_divide


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,0.65,0.7,0.65,0.65,None,0.6,None,0.65,0.65,0.65,0.6,0.6,0.65,0.65,None,None,0.6,None,None,None,None,0.6,0.7,0.7,0.7,0.7,0.65,0.65,0.65,0.6,0.6,None,0.65,0.65,0.65,0.6,0.65,0.65,0.75,0.75,0.7,0.65,0.65,None,None,0.65,None,0.6,0.6,None,None,None,None,0.6,None,0.65,0.65,0.65,0.6,None,0.65,0.65,0.6,None,None,0.75,0.65,0.6,0.65,0.65,0.7,0.65,0.6,None,None,None,0.7,0.6,0.7,0.65,0.7,0.7,None,None,None,None,None,0.8,0.65,0.7,0.7,0.65,0.65,None,None,None,None,None,0.6,0.7,0.7,0.7,0.65,0.6,0.65,0.65,0.65,0.65,0.65,None,None,0.65,0.6,0.6,0.65,None,None,None,None,0.65,0.75,None,0.7,0.65,0.65,0.65,None,None,None,None,0.7,0.7,0.7,0.65,0.7,0.65,0.7,0.65,0.75,None,0.6,None,0.7,0.65,0.7,0.65,0.65,0.65,0.7,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.666667,0.625,0.625,0.666667,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625,0.666667,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625,0.625,0.625,None,None,None,None,None,None,None,0.625,0.666667,0.708333,0.708333,0.708333
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.611111,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.611111,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,

In [420]:
print("Top ten highest accuracies")
topTen.reverse()
for top in topTen:
    print(
        f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
    )


Top ten highest accuracies
Accuracy 0.8 : subject 1, feature:-dataWCV-dataCV, kernellinear, C:0.5
Accuracy 0.75 : subject 5, feature:-dataFFTCV-dataCV, kernelrbf, C:1.62
Accuracy 0.75 : subject 5, feature:-dataFFTCV, kernelsigmoid, C:0.5
Accuracy 0.75 : subject 1, feature:-dataFFTCV-dataCV, kernellinear, C:0.5
Accuracy 0.75 : subject 1, feature:-dataCV, kernelsigmoid, C:1.62
Accuracy 0.75 : subject 1, feature:-dataCV, kernelsigmoid, C:0.5
Accuracy 0.75 : subject 8, feature:-dataCV, kernelsigmoid, C:0.5
Accuracy 0.75 : subject 8, feature:-dataCV, kernelrbf, C:1.62
Accuracy 0.75 : subject 1, feature:-dataFFTCV-dataWCV-dataCV, kernellinear, C:0.5
Accuracy 0.75 : subject 1, feature:-dataFFTCV-dataHRCV-dataCV, kernelsigmoid, C:1.62


In [421]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(maxInd)
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.65825243 0.65104167 0.61111111 0.         0.65336879
 0.63636364 0.625      0.63360656 0.625     ]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153
0,0.666667,0.658333,0.683333,0.647222,0.647222,0.666667,0.675,0.666667,0.658333,0.658333,0.658333,0.6,0.6,0.65,0.65,NaN,NaN,0.6,NaN,0.611111,NaN,NaN,0.6125,0.6625,0.65,0.65,0.7,0.65,0.641667,0.65,0.6,0.6125,NaN,0.65,0.641667,0.7,0.65,0.65,0.625,0.708333,0.675,0.641667,0.641667,0.641667,0.666667,NaN,0.65,0.625,0.6,0.6125,NaN,NaN,NaN,NaN,0.6,0.666667,0.65,0.65,0.65,0.6,NaN,0.6375,0.625,0.630556,NaN,0.6,0.729167,0.645833,0.666667,0.655556,0.658333,0.7,0.675,0.6,0.6,0.6,0.633333,0.655556,0.6125,0.7,0.6375,0.6625,0.7,0.633333,0.6125,0.625,0.633333,0.6,0.8,0.641667,0.675,0.675,0.65,0.65,0.7,0.65,0.6125,0.6125,0.6125,0.6,0.65,0.675,0.675,0.625,0.6,0.658333,0.655556,0.6375,0.625,0.625,0.666667,NaN,0.65,0.608333,0.625,0.655556,NaN,NaN,NaN,NaN,0.65,0.729167,NaN,0.7,0.658333,0.640341,0.641667,NaN,NaN,NaN,NaN,0.7,0.6625,0.655556,0.638889,0.641667,0.6375,0.6625,0.65,0.675,0.625,0.6125,NaN,0.7,0.625,0.65,0.625,0.65,0.65,0.675,0.633333,0.654167,0.708333,0.708333


Max average accuracy 0.8000000000000002
88
1.0
-dataWCV-dataCV
linear
C: 0.5


In [422]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(maxInd2)
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153
0,0.666667,1.316667,1.366667,1.941667,1.941667,0.666667,1.35,0.666667,1.316667,1.316667,1.316667,0.6,0.6,0.65,0.65,0.0,0.0,0.6,0.0,0.611111,0.0,0.0,1.225,1.325,1.3,1.3,0.7,0.65,1.925,1.3,0.6,1.225,0.0,0.65,1.925,1.4,1.3,1.3,1.25,2.125,2.025,1.925,1.925,1.925,0.666667,0.0,0.65,0.625,0.6,1.225,0.0,0.0,0.0,0.0,0.6,0.666667,0.65,0.65,0.65,0.6,0.0,1.275,1.875,1.891667,0.0,0.6,1.458333,2.583333,2.0,1.966667,1.316667,0.7,2.025,1.2,0.6,0.6,1.266667,1.311111,1.225,0.7,1.275,1.325,0.7,1.266667,1.225,0.625,1.266667,0.6,0.8,1.925,1.35,1.35,0.65,0.65,0.7,0.65,1.225,1.225,1.225,0.6,1.3,1.35,1.35,1.25,0.6,1.975,1.966667,1.275,1.25,1.25,0.666667,0.0,0.65,1.825,1.25,1.966667,0.0,0.0,0.0,0.0,0.65,1.458333,0.0,0.7,1.975,2.561364,1.925,0.0,0.0,0.0,0.0,0.7,1.325,1.966667,1.916667,1.925,1.275,1.325,1.3,2.025,0.625,1.225,0.0,0.7,1.25,1.3,1.25,0.65,0.65,2.025,1.266667,1.308333,0.708333,0.708333


67
4.0
Max total significant accuracy then averaged 0.6458333333333334 
-dataFFTCV-dataCV
rbf
C: 0.5 


In [423]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
